### Network Analysis Practice

This file will us a random sample file (N=5000) and explore the network of Twitter users who ...

In [31]:
%matplotlib inline

import pandas as pd
import json
import networkx as nx
import matplotlib.pyplot as plt
import gzip
import random
import numpy as np

from scipy import stats
from operator import itemgetter
import seaborn as sns

import re

In [9]:
users = []
with gzip.open('province_gender_available_metadata_added_with_year-220204.txt.gz', 'rb') as f:
    for l in f:
        user_object = json.loads(l)
        users.append(user_object)

In [11]:
len(users)

20000

In [35]:
users[1]

{'location': 'istanbul, türkiye',
 'description': 'karınca yükünü fil çekmez oldu..',
 'name': 'yılmaz şengül',
 'screen_name': 'deatime',
 'id_str': '31109031',
 'created_at': 'Tue Apr 14 12:22:59 +0000 2009',
 'province_codes': [{'source': 'location', 'pcode': 34}],
 'genders': [{'source': 'name', 'gender': 'male'}],
 'following': ['1481278402514636806',
  '15144122',
  '104272079',
  '1454407863129055236',
  '1311609086585253891',
  '2889618250',
  '2820162928',
  '930085513771540481',
  '1471854816783699968',
  '3665941462',
  '2870519824',
  '1468629567216963584',
  '603472906',
  '1273168800452665345',
  '1384949376993349638',
  '275713445',
  '1342739052',
  '952325509',
  '1447264979225305095',
  '1006935954664378368',
  '804618171181363200',
  '145961174',
  '1348605202249691137',
  '1139243727766118400',
  '281573829',
  '1455892203574865926',
  '212256405',
  '362369335',
  '1440803225356038145',
  '439630923',
  '518302854',
  '53333911',
  '1451478318465929250',
  '1450112

#### Sample generation from the raw data

In [4]:
'''
NSAMPLE = 1000
DATAFILE = "C:\\Users\\HP\\Desktop\\Özgem\\Koç\\Fall 2021\\Social Comquant\\20211007-user_objects_all.txt.gz"
def iterate_over_file(datafile):
    with gzip.open(datafile, 'r') as fl:
        for line in fl:
            try:
                dpoint = json.loads(line)
                yield dpoint # "yield" keyword will send object one by one
            except Exception as e:
                print('Error parsing data: {}'.format(e))
def sample_datastream(stream, nSample):
    samples = list()
    for c,item in enumerate(stream):
        if len(samples) < nSample:
            samples.append(item)
        else:
            r = random.randint(0,c+1)
            if r < nSample:
                samples[r] = item
    return samples, c

dataIterator = iterate_over_file(DATAFILE)
mySample, count = sample_datastream(dataIterator, NSAMPLE)
print('{} items processed and {} samples collected'.format(count, len(mySample)))

with open("sample_1000.txt", "w") as random_sample:
    for i in mySample:
        random_sample.write(json.dumps(i))
        random_sample.write("\n")
'''

#### Cleaning the raw data

In [12]:
pd.set_option('display.float_format', lambda x: '%.f' % x)

In [14]:
tweets_df = pd.read_json("province_gender_available_metadata_added_with_year-220204.txt.gz", lines=True)

In [17]:
tweets_df.head(20)

,location,description,name,screen_name,id_str,created_at,province_codes,genders,following,followers,tweets
0,"mersin, türkiye","• gez 👣 gör 👀 yaşa 🤸‍♂️ | mühendis, gezgin, fo...",furkan,furkanozcantr,1319722270185693184,2020-10-23 19:28:17+00:00,"[{'source': 'location', 'pcode': 33}, {'source...","[{'source': 'name', 'gender': 'male'}, {'sourc...",[],[],[]
1,"istanbul, türkiye",karınca yükünü fil çekmez oldu..,yılmaz şengül,deatime,31109031,2009-04-14 12:22:59+00:00,"[{'source': 'location', 'pcode': 34}]","[{'source': 'name', 'gender': 'male'}]","[1481278402514636806, 15144122, 104272079, 145...","[1480050379, 2878337056, 1320285286585217024, ...","[{'ref_twt_id_str': '1481612518636277761', 're..."
2,"tunceli, türkiye",arbeitlos / asosial\nişsizzz\n bêrlin-tunceli-...,şenayşahin_aslan_karabulut,happymoontr,779327836364693504,2016-09-23 14:33:30+00:00,"[{'source': 'location', 'pcode': 62}, {'source...","[{'source': 'name', 'gender': 'female'}]",[],[],[]
3,gaziantep,hayat bana acımasızlığı öğretti ama ben yinede...,serkan35,serkan317015780,1259649574123208704,2020-05-11 01:00:54+00:00,"[{'source': 'location', 'pcode': 27}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1452303769388060685, 1477289257597296641, 144...","[1479883851841347597, 1460835887944773632, 135...","[{'ref_twt_id_str': '1479320229914288128', 're..."
4,bursa'sında...,tony (58) was a lecturer at the university fo...,tuncay bayer,tuncaybayer1,305017942,2011-05-25 14:11:00+00:00,"[{'source': 'location', 'pcode': 16}]","[{'source': 'name', 'gender': 'male'}, {'sourc...",[],[],[]
5,,ankaradan bayanlar dm,ali haydar,boran12084165,1106194712451534848,2019-03-14 14:05:46+00:00,"[{'source': 'description', 'pcode': 6}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1468769999112318981, 1099373218949943296, 127...","[1099373218949943296, 1457313754304106499, 143...","[{'twt_id_str': '1449010683568394240', 'twt_tx..."
6,,evli❤mutlu❤çocuklu,cilem büşra,busracilem,1138203804149714944,2019-06-10 21:58:28+00:00,"[{'source': 'description', 'pcode': 33}]","[{'source': 'name', 'gender': 'female'}, {'sou...","[108867102, 417157710, 842366096, 2317354298, ...","[1144212583706243074, 1117857832362287104, 802...","[{'twt_id_str': '1138368360058671104', 'twt_tx..."
7,,trt istanbul radyosu thm /çatalca belediyesi t...,mehmet şerif aydın,mehmetserifaydn,600206894,2012-06-05 15:54:15+00:00,"[{'source': 'description', 'pcode': 34}, {'sou...","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1178885521, 210507380, 1189571058229268480, 1...","[2850080260, 2676419056, 3770444417, 319866133...","[{'twt_id_str': '1433563181834661896', 'twt_tx..."
8,istanbul,graphic artist,asım demirağ,ademirag,25269500,2009-03-19 11:00:10+00:00,"[{'source': 'location', 'pcode': 34}]","[{'source': 'name', 'gender': 'male'}]","[204956039, 1346065229944381441, 1291027909558...","[1434961851583012864, 790627, 31164573, 117674...","[{'twt_id_str': '1438136219041402887', 'twt_tx..."
9,"kadıköy, istanbul","canınızın, malınızın ve ailenizin güvenliğinde...",the zazza ( serdar ),serdar11594743,1081300621272207360,2019-01-04 21:25:32+00:00,"[{'source': 'location', 'pcode': 34}, {'source...","[{'source': 'screen_name', 'gender': 'male'}]",[],[],[]


In [ ]:
#Retweeted status'u olmayanları elediğin bir kod yaz.

In [16]:
tweets_df.columns

Index(['location', 'description', 'name', 'screen_name', 'id_str',
       'created_at', 'province_codes', 'genders', 'following', 'followers',
       'tweets'],
      dtype='object')

#### Creating the graph

In [37]:
tweets_final = pd.DataFrame(columns = ["created_at", "id_str", "location", "in_reply_to_status_id", "in_reply_to_user_id",
                                      "retweeted_id", "fav_id", "user_mentions_id","tweets", "screen_name"])

In [38]:
equal_columns = ['created_at', 'id_str', 'tweets', 'screen_name', 'location']
tweets_final[equal_columns] = tweets_df[equal_columns]

In [49]:
def get_usermentions(tweets_final):
    tweets_final["user_mentions_id"] = tweets_df["tweets"].apply(lambda x: x['ref_usr_id_str'] if x["type"] == "quote" else np.nan)
    return tweets_final
    

In [40]:
def get_retweets(tweets_final):
    tweets_final["retweeted_id"] = tweets_df["tweets"].apply(lambda x: x['ref_usr_id_str'] if x["type"] == "retweet" else np.nan)
    return tweets_final

In [41]:
def get_in_reply(tweets_final):
    tweets_final["in_reply_to_status_id"] = tweets_df["tweets"].apply(lambda x: x["ref_twt_id_str"] if x["type"] == "reply" else np.nan)
    tweets_final["in_reply_to_user_id"] = tweets_df["tweets"].apply(lambda x: x["ref_usr_id_str"] if x["type"] == "reply" else np.nan)


In [42]:
def get_fav(tweets_final):
    tweets_final["fav_id"] = tweets_df["tweets"].apply(lambda x: x["ref_usr_id_str"] if x["type"] == "fav" else np.nan)
    return tweets_final

In [50]:
#the creation of the dataframe that we will use
def fill_df(tweets_final):
    get_usermentions(tweets_final)
    get_retweets(tweets_final)
    get_fav(tweets_final)
    return tweets_final

#### Interaction between the users

In [46]:
def get_interactions(row):
    user = row["id_str"], row["screen_name"]
    # Be careful if there is no user id
    if user[0] is None:
        return (None, None), []
    
    interactions = set()
    
    interactions.add((row["in_reply_to_user_id"]))
    interactions.add((row["retweeted_id"]))
    interactions.add((row["user_mentions_id"]))
    
    interactions.discard((row["id_str"], row["screen_name"]))
    interactions.discard((None, None))
    return user, interactions

In [51]:
tweets_final = fill_df(tweets_final)

TypeError: list indices must be integers or slices, not str

In [47]:
tweets_final = fill_df(tweets_final)
tweets_final = tweets_final.where((pd.notnull(tweets_final)), None)

TypeError: list indices must be integers or slices, not str

#### Visualization

In [52]:
graph = nx.Graph()

In [54]:
for index, tweet in tweets_final.iterrows():
    user, interactions = get_interactions(tweet)
    user_id, user_name = user
    tweet_id = tweet["tweets"]["twt_id_str"]
    for interaction in interactions:
        int_id = interaction
        graph.add_edge(user_id, int_id, tweet_id_=tweet_id)
        

        graph.node[user_id]["id"] = user_id
        graph.node[int_id]["id"] = int_id 

TypeError: list indices must be integers or slices, not str

In [ ]:
print(f"There are {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges present in the Graph")

In [ ]:
degrees = [val for (node, val) in graph.degree()]

In [ ]:
print(f"The maximum degree of the Graph is {np.max(degrees)}")   
print(f"The minimum degree of the Graph is {np.min(degrees)}")  
print(f"The average degree of the nodes in the Graph is {np.mean(degrees):.1f}")  
print(f"The most frequent degree of the nodes found in the Graph is {stats.mode(degrees)[0][0]}") 

In [ ]:
if nx.is_connected(graph):
    print("The graph is connected")
else:
    print("The graph is not connected")

In [ ]:
print(f"There are {nx.number_connected_components(graph)} connected components in the Graph")  

In [ ]:
largest_subgraph = max(nx.connected_component_subgraphs(graph), key=len)
print(f"There are {largest_subgraph.number_of_nodes()} nodes and {largest_subgraph.number_of_edges()} \
edges present in the largest component of the Graph")

In [ ]:
if nx.is_connected(largest_subgraph):
    print("The graph is connected")
else:
    print("The graph is not connected")

In [ ]:
print(f"The average clustering coefficient is {nx.average_clustering(largest_subgraph)} in the largest subgraph")
print(f"The transitivity of the largest subgraph is {nx.transitivity(largest_subgraph)}")

In [ ]:
print(f"The diameter of our Graph is {nx.diameter(largest_subgraph)}")
print(f"The average distance between any two nodes is {nx.average_shortest_path_length(largest_subgraph):.2f}")

In [ ]:
graph_centrality = nx.degree_centrality(largest_subgraph)
max_de = max(graph_centrality.items(), key=itemgetter(1))
graph_closeness = nx.closeness_centrality(largest_subgraph)
max_clo = max(graph_closeness.items(), key=itemgetter(1))
graph_betweenness = nx.betweenness_centrality(largest_subgraph, normalized=True, endpoints=False)
max_bet = max(graph_betweenness.items(), key=itemgetter(1))

In [ ]:
print(f"the node with id {max_de[0]} has a degree centrality of {max_de[1]:.2f} which is the maximum of the Graph")
print(f"the node with id {max_clo[0]} has a closeness centrality of {max_clo[1]:.2f} which is the maximum of the Graph")
print(f"the node with id {max_bet[0]} has a betweenness centrality of {max_bet[1]:.2f} which is the maximum of the Graph")

In [ ]:
node_and_degree = largest_subgraph.degree()
colors_central_nodes = ['orange', 'red']
central_nodes = ['393852070', '2896294831']

In [ ]:
pos = nx.spring_layout(largest_subgraph, k=0.05)